## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-15-20-52-20 +0000,nyt,Supreme Court Appears Skeptical of Key Provisi...,https://www.nytimes.com/2025/10/15/us/politics...
1,2025-10-15-20-46-21 +0000,nyt,"Jeep Parent Stellantis, Faced With Trump’s Tar...",https://www.nytimes.com/2025/10/15/world/canad...
2,2025-10-15-20-44-13 +0000,startribune,Minnesota AG goes after two contractors for al...,https://www.startribune.com/minnesota-attorney...
3,2025-10-15-20-42-54 +0000,nypost,Zohran Mamdani backs NY class size law with $1...,https://nypost.com/2025/10/15/us-news/zohran-m...
4,2025-10-15-20-41-16 +0000,wapo,U.S. pushes Russia to negotiate as Hegseth war...,https://www.washingtonpost.com/world/2025/10/1...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2318/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
13,trump,52
172,hamas,21
313,gaza,18
110,he,16
36,new,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
272,2025-10-15-09-30-00 +0000,wsj,Governors across 15 states are forming a new p...,https://www.wsj.com/health/healthcare/rfk-jr-h...,122
43,2025-10-15-19-58-31 +0000,latimes,California judge halts Trump federal job cuts ...,https://www.latimes.com/politics/story/2025-10...,106
74,2025-10-15-19-21-00 +0000,wsj,Israel and Hamas Start Next Phase of Talks on ...,https://www.wsj.com/world/middle-east/israel-a...,98
18,2025-10-15-20-27-34 +0000,cbc,Judge blocks Trump from firing workers during ...,https://www.cbc.ca/news/world/trump-shutdown-9...,98
273,2025-10-15-09-24-36 +0000,bbc,Trump demands Hamas disarm as group moves to a...,https://www.bbc.com/news/articles/cjr034p5prlo...,96


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
272,122,2025-10-15-09-30-00 +0000,wsj,Governors across 15 states are forming a new p...,https://www.wsj.com/health/healthcare/rfk-jr-h...
155,68,2025-10-15-16-07-50 +0000,nypost,US student group calls for ‘death’ to Zionist ...,https://nypost.com/2025/10/15/world-news/stude...
263,67,2025-10-15-10-00-00 +0000,nypost,Dr. Keith Ablow tells ‘Pod Force One’ why he f...,https://nypost.com/2025/10/15/us-news/dr-keith...
206,50,2025-10-15-13-12-06 +0000,nyt,The case will test what is left of the Voting ...,https://www.nytimes.com/live/2025/10/15/us/sup...
43,49,2025-10-15-19-58-31 +0000,latimes,California judge halts Trump federal job cuts ...,https://www.latimes.com/politics/story/2025-10...
45,44,2025-10-15-19-57-27 +0000,cbc,Finance minister says world economy showing si...,https://www.cbc.ca/news/politics/champagne-wor...
47,38,2025-10-15-19-52-00 +0000,wsj,SL Green’s deal to buy the 36-story building ...,https://www.wsj.com/real-estate/commercial/sl-...
353,37,2025-10-14-22-31-05 +0000,nypost,Trump honors late Charlie Kirk with Medal of F...,https://nypost.com/2025/10/14/us-news/trump-ho...
71,37,2025-10-15-19-28-40 +0000,nypost,Supreme Court seems set to restrict race-based...,https://nypost.com/2025/10/15/us-news/supreme-...
291,35,2025-10-15-07-32-43 +0000,wsj,Stock Market Today: S&P 500 Finishes Higher Af...,https://www.wsj.com/livecoverage/stock-market-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
